# In this notebook we will map fMRI data to a hippocampal surface, and then visualize it

We will assume:
- HippUnfold was run on the native T1w image from the `ds002168` dataset
- fMRI data is already preprocessed and registered to T1w space (eg. via fMRIprep)

In [ ]:
# set up a test subject

func_dir = './ds002168'
hippunfold_dir = './ds002168_hippunfold/hippunfold'

subject = '1425'
hemi = ['L','R']
label = ['hipp','dentate'] #the dentate gyrus is given a separate surface from the rest of the hippocampus. Here we will include it but this is optional.

!mkdir -p tmp # make a temporary directory for storing files

## 1) Sample data along a hippocampal surface:

NOTE this assumes we have a few tools (`wb_command`) already installed and in out `PATH`

In [ ]:
# make sure we have and understand wb_command:

!wb_command -volume-to-surface-mapping

## Sampling density

For fMRI data, resolution is typically >=2mm, and so I recommend using `den-2mm` surfaces (obtained by adding the flag `--output_density 2mm ` when executing HippUnfold). 

Otherwise the default `den-0p5mm` surfaces will work fine, but you may be unnecessarily oversampling the fMRI data.

In [ ]:
# now we can construct our call to wb_command, looping over hemispheres and surface label files

for h in range(len(hemi)):
    for l in range(len(label)):
        cmd = f'wb_command -volume-to-surface-mapping '\
            f'{func_dir}/sub-{subject}/func/sub-{subject}_space-T1w_task-mst_run-1_bold.nii.gz '\
            f'{hippunfold_dir}/sub-{subject}/surf/sub-{subject}_hemi-{hemi[h]}_space-T1w_den-2mm_label-{label[l]}_midthickness.surf.gii '\
            f'tmp/sub-{subject}_hemi-{hemi[h]}_space-T1w_den-2mm_label-{label[l]}_task-mst_run-1_bold.func.gii '\
            f'-enclosing'
        !echo {cmd}
        !{cmd}

## 2) (optional) Smooth data along hippocampal surface:

Most of the time fMRI data benefits from smoothing. This is best done along a surface so values are not combined across a suclus (or across hippocampal folds).

Note that depending on your data resolution you may want to adjust the size of the smoothing kernel, but remember that the hippocampus is only ~10mm wide in some regions!

In [ ]:
sigma = 2 #Gaussian smoothing kernal sigma (mm)

for h in range(len(hemi)):
    for l in range(len(label)):
        cmd = f'wb_command -metric-smoothing '\
            f'{hippunfold_dir}/sub-{subject}/surf/sub-{subject}_hemi-{hemi[h]}_space-T1w_den-2mm_label-{label[l]}_midthickness.surf.gii '\
            f'tmp/sub-{subject}_hemi-{hemi[h]}_space-T1w_den-2mm_label-{label[l]}_task-mst_run-1_bold.func.gii '\
            f'{sigma} '\
            f'tmp/sub-{subject}_hemi-{hemi[h]}_space-T1w_den-2mm_label-{label[l]}_task-mst_run-1_bold_smooth-{sigma}mm.func.gii'
        !echo {cmd}
        !{cmd}

## 3) Load and understand surface data

Here, we will only load one hemisphere and one surface (ie. no dentate gyrus). 

This is to showcase some of the plotting tools in the `HippUnfold_toolbox` and to illustrate how surface data is formatted.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import sys
sys.path.insert(1, '/data/mica3/opt/hippunfold_toolbox')
from hippunfold_toolbox import plotting
np.set_printoptions(threshold=10)


surface = nib.load(f'{hippunfold_dir}/sub-{subject}/surf/sub-{subject}_hemi-R_space-T1w_den-2mm_label-hipp_midthickness.surf.gii')
func = nib.load(f'tmp/sub-{subject}_hemi-R_space-T1w_den-2mm_label-hipp_task-mst_run-1_bold_smooth-{sigma}mm.func.gii')

### Let's look at how "metric" data is formatted

In [ ]:
func

In [ ]:
len(func.darrays)

In [ ]:
func.darrays[0]

In [ ]:
func.darrays[0].data

In [ ]:
func.darrays[0].data.shape

### Let's look at how "surface" data is formatted

In [ ]:
surface

In [ ]:
surface.get_arrays_from_intent('NIFTI_INTENT_POINTSET')[0].data

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.scatter(surface.get_arrays_from_intent('NIFTI_INTENT_POINTSET')[0].data[:,0],
           surface.get_arrays_from_intent('NIFTI_INTENT_POINTSET')[0].data[:,1],
           surface.get_arrays_from_intent('NIFTI_INTENT_POINTSET')[0].data[:,2])

In [ ]:
surface.get_arrays_from_intent('NIFTI_INTENT_TRIANGLE')[0].data

### Now let's use a HippUnfold_toolbox function to make plotting easier

In [ ]:
# plot only the very first timepoint

fig,ax = plotting.plot_gifti(surface,func.darrays[0].data)
ax.view_init(elev=90, azim=-90)

In [ ]:
# also plot on an unfolded surface

surface_unfolded = nib.load(f'{hippunfold_dir}/sub-{subject}/surf/sub-{subject}_hemi-L_space-unfolded_den-2mm_label-hipp_midthickness.surf.gii')
fig,ax = plotting.plot_gifti(surface_unfolded,func.darrays[0].data)
ax.view_init(elev=-90, azim=0)

## 4) Make a nice figure with all data

This function will use generic surface (ie. not subject-specific) to plot L+R, dentate+hipp, and folded+unfolded surfaces.

In [ ]:
# load the hippocampal functional data

hipp_func = np.array([])
for h in range(len(hemi)):
    for l in range(len(label)):
        f = nib.load(f'tmp/sub-{subject}_hemi-{hemi[h]}_space-T1w_den-2mm_label-{label[l]}_task-mst_run-1_bold_smooth-{sigma}mm.func.gii')
        # format into a Vxt matric (vertices x timepoints)
        fvol = np.zeros((len(f.darrays[0].data),len(f.darrays)))
        for t in range(len(f.darrays)):
            fvol[:,t] = f.darrays[t].data
        hipp_func = np.vstack((hipp_func,fvol)) if hipp_func.size else fvol

hipp_func.shape

In [ ]:
# plot (only the first timepoint)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,8), subplot_kw={'projection': "3d"})
plotting.surfplot_canonical_foldunfold(ax, hipp_func[:,0], den='2mm')

### Let's have some fun and turn this data into a gif file:

In [ ]:
# make a gif (optional, takes time!)

import imageio
with imageio.get_writer('tmp/Hippo_ts.gif', mode='I') as writer:
    for t in range(hipp_func.shape[1]):
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,8), subplot_kw={'projection': "3d"})
        plotting.surfplot_canonical_foldunfold(ax, hipp_func[:,t], den='2mm', cwindow=[150,350])
        plt.savefig(f'tmp/frame_{t}.png', dpi=fig.dpi)
        image = imageio.imread(f'tmp/frame_{t}.png')
        writer.append_data(image)
        
!rm tmp/frame_*.png

## Scaling up

This code can be looped over all subjects and runs, and can be run on different features. For example, instead of an fMRI image, we could transform and sample a DWI image. 

All hippunfold subjects have corresponding vertices, meaning that we could load and average across subjects WHERE APPROPRIATE - we wouldn't do this for rsfMRI since subjects may be thinking of different things at different times. We could consider doing this for a time-locked experiment like movie watching, or, we could average across subjects for each vertex for a structural measure like Fractional Anisotropy (FA) derived from DWI. 

We could consider running an expierment where we compare FA measures across two groups at each vertex. This is more precise than other methods since vertices are carefully aligned between subjects according to their topology in HippUnfold. We could also find the subset of vertices belonging to a given subfield using (for example) the `sub-HC002_hemi-L_space-T1w_den-0p5mm_label-hipp_subfields.label.gii` HippUnfold output file. Averaging data within one of these ROIs may help improve signal-to-noise ratio, at the cost of no longer being able to check for anterior-posterior differences. Alternatively, we could parametrically test whether data aligns with the anterior-posterior axis by correlating it with the unfolded y axis. 